# TRAINING A CHATBOT INTENT CLASSIFIER


## Importing the libraries

In [1]:
import nltk
import numpy as np
import random

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
# download the wordnet and stopwords corpus
nltk.download('wordnet') # wordnet is a lexical database for the English language
nltk.download('stopwords') # stopwords means words like 'the', 'a', 'an', 'is', 'are', etc.
nltk.download('punkt') # punkt means punctuations

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dagbo_b40tnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dagbo_b40tnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dagbo_b40tnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Importing the json file

In [3]:
# import intents file
import json

def load_intents():
    with open('intents.json') as file:
        intents = json.load(file)
    return intents

intents = load_intents()

## Preprocessing the data

In [4]:
# initialize lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer() # lemma is the root form of the word and it is very accurate than stemming


# loop through each sentence in the intent's patterns
def preprocess_intents(intents):
    # create empty lists for documents, classes and words
    documents = [] # documents means patterns
    classes = [] # classes means tags
    words = [] # words means vocabulary
    ignore_letters = ['!', '?', ',', '.'] # ignore these letters
    stop_words = set(stopwords.words('english')) # stop words are words like 'the', 'a', 'an', 'is', 'are', etc.


    for intent in intents['intents']:
        
        # debug for keyerror @ 'patterns'
        # print(intent['patterns'])
        for pattern in intent['patterns']:
            # tokenize each and every word in the sentence
            word = nltk.word_tokenize(pattern) 
            # lemmatize each word and convert into lowercase
            word = [lemmatizer.lemmatize(w.lower()) for w in word if w not in stop_words and w not in ignore_letters]
            # add word to the word list
            words.extend(word) # extend means add to the list and append means add to the end of the list
            # add word(s) to documents
            documents.append((word, intent['tag'])) #
            # add tags to our classes list
            if intent['tag'] not in classes: # if tag is not in classes list
                classes.append(intent['tag']) # then add it to the classes list
                
    # sort words and remove duplicates
    words = sorted(list(set(words)))
    # sort classes
    classes = sorted(list(set(classes)))
    return documents, classes, words

documents, classes, words = preprocess_intents(intents)


## save the preprocessed data

In [5]:
import pickle

#save to file using pickle to dump
pickle.dump(words, open ('words.pkl', 'wb'))#wb means writing into binariries
pickle.dump(classes, open ('classes.pkl', 'wb'))


## Create training and testing data

In [9]:

#represents the words as numerical values using bag_of_words
train = []
empty_output = [0] * len(classes)

for doc in documents:
    #create an empty list of bag of words
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns  else bag.append(0)
    
    output = list(empty_output)
    output[classes.index(doc[1])] = 1
    # Ensure the 'bag' has the same length as 'words'
    assert len(bag) == len(words), f"Bag length: {len(bag)}, expected: {len(words)}"
    
    # Ensure the 'output' has the same length as 'classes'
    assert len(output) == len(classes), f"Output length: {len(output)}, expected: {len(classes)}"
    train.append([bag, output])

In [11]:
# shuffle the features and make numpy array
random.shuffle(train)
# Before converting to a NumPy array, ensure all elements in 'train' have the same length
#assert all(len(item) == len(train[0]) for item in train), "Not all lists in 'train' have the same length."
train = np.array(train, dtype=object)

# create train and test lists. X - patterns, Y - intents
train_x = list(train[:,0])
train_y = list(train[:,1])



## Building the model using long short term memory (LSTM)

In [13]:
# import tensorflow libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay # learning rate decay over each update

# Create a learning rate schedule
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

# Build the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu')) # input layer
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu')) # hidden layer
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax')) # output layer

# Compile the model
sgd = SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True) # stochastic gradient descent
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])



## Training and saving the model

In [16]:
# Training and saving the model
solentBot = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
loss, accuracy = model.evaluate(np.array(train_x), np.array(train_y))
print(f"Loss: {loss}, accuracy: {accuracy}")
#model.save('solentBot_model.h5', solentBot)

Epoch 1/200
53/53 [==============================] - 0s 2ms/step - loss: 0.1772 - accuracy: 0.9468
Epoch 2/200
53/53 [==============================] - 0s 2ms/step - loss: 0.3688 - accuracy: 0.8935
Epoch 3/200
53/53 [==============================] - 0s 2ms/step - loss: 0.2171 - accuracy: 0.9392
Epoch 4/200
53/53 [==============================] - 0s 2ms/step - loss: 0.2029 - accuracy: 0.9278
Epoch 5/200
53/53 [==============================] - 0s 2ms/step - loss: 0.1841 - accuracy: 0.9392
Epoch 6/200
53/53 [==============================] - 0s 2ms/step - loss: 0.2127 - accuracy: 0.9316
Epoch 7/200
53/53 [==============================] - 0s 2ms/step - loss: 0.2366 - accuracy: 0.9240
Epoch 8/200
53/53 [==============================] - 0s 2ms/step - loss: 0.2891 - accuracy: 0.9049
Epoch 9/200
53/53 [==============================] - 0s 2ms/step - loss: 0.2062 - accuracy: 0.9316
Epoch 10/200
53/53 [==============================] - 0s 2ms/step - loss: 0.2531 - accuracy: 0.9202
Epoch 11/

In [19]:
# save the model to disk
model.save('solentBot_model.keras', solentBot)
print("model created, saved and trained")

model created, saved and trained
